Assignment: Flowers Recognition <br>
Dataset Description:<br>

This dataset contains 4242 images of flowers.<br>
The data collection is based on the data flicr, google images, yandex images.<br>
You can use this datastet to recognize plants from the photo.<br>

Attribute Information:<br>
The pictures are divided into five classes: chamomile, tulip, rose, sunflower, dandelion.<br>
For each class there are about 800 photos. Photos are not high resolution, about 320x240 pixels. <br>
<b>Also explore how to resize images in tensorflow and then resize all the images to a same size. </b> <br>
This is a Multiclass Classification Problem.<br>




WORKFLOW : <br>
Load Data <br>
Split into 60 and 40 ratio.<br>
Encode labels.<br>
Create Model<br>
Compilation Step (Note : Its a Multiclass Classification problem , select loss , metrics according to it)<br>
Train the Model.<br>
If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .<br>
Prediction should be > 85%<br>
Evaluation Step<br>
Prediction<br>




Data : <br>
https://drive.google.com/file/d/1-OX6wn5gA-bJpjPNfSyaYQLz-A-AB_uj/view?usp=sharing

In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow import keras
#from tensorflow.keras import layers, Dense, Input, InputLayer, Flatten
from tensorflow.keras.models import Sequential, Model
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import struct
import cv2
import imageio

IMAGE_HEIGHT = 320
IMAGE_WIDTH = 240

In [2]:
# function to load flower images from local folder named flowers with sub folders named after each class of flowers

def create_dataset(img_folder):
   
    image_array=[]
    class_array=[]
    label = 0
    
    for sub_folder in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, sub_folder)):
       
            image_path= os.path.join(img_folder, sub_folder,  file)
           # image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
            image= cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image= cv2.resize(image, (IMAGE_HEIGHT, IMAGE_WIDTH), interpolation = cv2.INTER_AREA)
            image = image.astype('float32')
            image /= 255 
            image = image.reshape(IMAGE_HEIGHT*IMAGE_WIDTH*1)
            image_array.append(image)
            class_array.append(label)
            
        label += 1
       
    return image_array, class_array

In [3]:
#image_data, class_name = create_dataset('flowers/') ; The data set is stored under local hard disk in subfolder 'flowers'
images_data, class_data = create_dataset('flowers/')

from sklearn.utils import shuffle
x, y = shuffle(images_data, class_data)
x = np.array(x)
y = np.array(y)

In [4]:
# converting'y' to categorical values from non-numeric

from keras.utils import to_categorical

y = to_categorical(y).astype('float32')

In [ ]:
from sklearn.model_selection import train_test_split

# Defines ratios, w.r.t. whole dataset.
ratio_train = 0.6 # 60%
ratio_test  = 0.4  # 40%
# Produces 40% test split.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=ratio_test, random_state=42)

In [ ]:
from keras import models
from keras import layers

model=models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(IMAGE_HEIGHT*IMAGE_WIDTH,)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

In [ ]:
from keras import optimizers
from keras import losses
from keras import metrics

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,
                    y_train,
                    epochs=30,
                    batch_size=128)

In [ ]:
history_dict=history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
history = model.fit(x=np.array(image_data, np.float32), y=np.array(list(map(int,target_val)), np.float32), epochs=5)
#history = model.fit(x=tf.cast(np.array(image_data), tf.float64), y=tf.cast(list(map(int,target_val)),tf.int32), epochs=5)